# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from hidden import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol  = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2614310453300,
 'week52high': 199.11,
 'week52low': 162.79,
 'week52highSplitAdjustOnly': 199.62,
 'week52lowSplitAdjustOnly': 164.08,
 'week52change': 0.007046949325849594,
 'sharesOutstanding': 15441881000,
 'float': 0,
 'avg10Volume': 50367857,
 'avg30Volume': 55605867,
 'day200MovingAvg': 181.33,
 'day50MovingAvg': 170.81,
 'employees': 164000,
 'ttmEPS': 6.42,
 'ttmDividendRate': 0.9562637739139714,
 'dividendYield': 0.005648338889037043,
 'nextDividendDate': '',
 'exDividendDate': '2024-02-09',
 'nextEarningsDate': '2024-05-02',
 'peRatio': 25.906577480602103,
 'beta': 1.031058836813742,
 'maxChangePercent': 65.34272502841021,
 'year5ChangePercent': 2.4611272276023364,
 'year2ChangePercent': 0.09775391638136743,
 'year1ChangePercent': 0.04345210690422618,
 'ytdChangePercent': -0.11841099439907188,
 'month6ChangePercent': 0.019712432661636248,
 'month3ChangePercent': -0.11790711087746096,
 'month1ChangePercent': -0.0024158859230452023,
 '

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.04345210690422618

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])



my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,price,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
        
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe._append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'

            ],index=my_columns
            ),ignore_index=True
            )
            
        except KeyError:
            print(f"Symbol '{symbol}' not found in data")
            
final_dataframe

Symbol 'ABC' not found in data
Symbol 'ANTM' not found in data
Symbol 'DISCA' not found in data
Symbol 'FBHS' not found in data
Symbol 'HFC' not found in data
Symbol 'NLOK' not found in data
Symbol 'RE' not found in data
Symbol 'VIAC' not found in data
Symbol 'WLTW' not found in data


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,137.74,0.042219,N/A
1,AAL,13.88,0.089482,N/A
2,AAP,75.14,-0.376010,N/A
3,AAPL,169.30,0.043452,N/A
4,ABBV,159.62,0.026571,N/A
...,...,...,...,...
491,YUM,141.77,0.046991,N/A
492,ZBH,119.35,-0.135226,N/A
493,ZBRA,297.21,0.059874,N/A
494,ZION,41.76,0.666082,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,270,LB,79.92,2.281287,N/A
1,339,NVDA,877.35,2.255945,N/A
2,406,SLG,50.61,1.415602,N/A
3,204,GPS,21.35,1.390068,N/A
4,384,RCL,140.59,1.360477,N/A
5,334,NRG,72.81,1.263802,N/A
6,48,AVGO,1344.07,1.256092,N/A
7,471,WDC,71.36,1.183599,N/A
8,194,GE,162.35,1.127766,N/A
9,190,FTI,26.59,1.090738,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
                           
    except ValueError:
        print("That's not a number...\n Please try again...")
        portfolio_size = input('Enter the size of your portfolio: ')
        
        
portfolio_input()
print(portfolio_size)
        

Enter the size of your portfolio: 10000000
10000000


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy']= math.floor(position_size/final_dataframe.loc[i,'Price'])
    
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,270,LB,79.92,2.281287,2502
1,339,NVDA,877.35,2.255945,227
2,406,SLG,50.61,1.415602,3951
3,204,GPS,21.35,1.390068,9367
4,384,RCL,140.59,1.360477,1422
5,334,NRG,72.81,1.263802,2746
6,48,AVGO,1344.07,1.256092,148
7,471,WDC,71.36,1.183599,2802
8,194,GE,162.35,1.127766,1231
9,190,FTI,26.59,1.090738,7521


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,price,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_dataframe = hqm_dataframe._append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'HQM Score'

            ],index=hqm_columns
            ),ignore_index=True
            )
        except KeyError:
            print(f"Symbol '{symbol}' not found in data")
            
            
hqm_dataframe

Symbol 'ABC' not found in data
Symbol 'ANTM' not found in data
Symbol 'DISCA' not found in data
Symbol 'FBHS' not found in data
Symbol 'HFC' not found in data
Symbol 'NLOK' not found in data
Symbol 'RE' not found in data
Symbol 'VIAC' not found in data
Symbol 'WLTW' not found in data


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.74,N/A,0.042219,N/A,0.324869,N/A,0.053239,N/A,-0.044639,N/A,HQM Score
1,AAL,13.88,N/A,0.089482,N/A,0.244843,N/A,-0.082617,N/A,-0.069705,N/A,HQM Score
2,AAP,75.14,N/A,-0.376010,N/A,0.483286,N/A,0.137838,N/A,-0.105261,N/A,HQM Score
3,AAPL,169.30,N/A,0.043452,N/A,0.019712,N/A,-0.117907,N/A,-0.002416,N/A,HQM Score
4,ABBV,159.62,N/A,0.026571,N/A,0.120179,N/A,-0.020008,N/A,-0.100894,N/A,HQM Score
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,141.77,N/A,0.046991,N/A,0.205503,N/A,0.103722,N/A,0.051706,N/A,HQM Score
492,ZBH,119.35,N/A,-0.135226,N/A,0.161878,N/A,-0.017385,N/A,-0.064875,N/A,HQM Score
493,ZBRA,297.21,N/A,0.059874,N/A,0.451008,N/A,0.178610,N/A,0.020008,N/A,HQM Score
494,ZION,41.76,N/A,0.666082,N/A,0.402736,N/A,-0.041349,N/A,0.013838,N/A,HQM Score


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
      for time_period in time_periods:
    # Filter out rows with missing values (optional)
        price_returns = hqm_dataframe[f'{time_period} Price Return'].dropna()

    # Check for empty column
        if len(price_returns) > 0:
              percentile = stats.percentileofscore(price_returns, hqm_dataframe.loc[row, f'{time_period} Price Return']) / 100
        else:
              percentile = np.nan  # Or assign a specific value

        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = percentile
hqm_dataframe
        

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.74,N/A,0.042219,0.365854,0.324869,0.743902,0.053239,0.593496,-0.044639,0.353659,HQM Score
1,AAL,13.88,N/A,0.089482,0.434959,0.244843,0.621951,-0.082617,0.162602,-0.069705,0.21748,HQM Score
2,AAP,75.14,N/A,-0.376010,0.018293,0.483286,0.896341,0.137838,0.823171,-0.105261,0.101626,HQM Score
3,AAPL,169.30,N/A,0.043452,0.371951,0.019712,0.184959,-0.117907,0.105691,-0.002416,0.611789,HQM Score
4,ABBV,159.62,N/A,0.026571,0.337398,0.120179,0.367886,-0.020008,0.315041,-0.100894,0.115854,HQM Score
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,141.77,N/A,0.046991,0.373984,0.205503,0.575203,0.103722,0.754065,0.051706,0.896341,HQM Score
492,ZBH,119.35,N/A,-0.135226,0.130081,0.161878,0.471545,-0.017385,0.325203,-0.064875,0.235772,HQM Score
493,ZBRA,297.21,N/A,0.059874,0.398374,0.451008,0.871951,0.178610,0.894309,0.020008,0.756098,HQM Score
494,ZION,41.76,N/A,0.666082,0.932927,0.402736,0.835366,-0.041349,0.256098,0.013838,0.707317,HQM Score


# Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.74,N/A,0.042219,0.365854,0.324869,0.743902,0.053239,0.593496,-0.044639,0.353659,0.514228
1,AAL,13.88,N/A,0.089482,0.434959,0.244843,0.621951,-0.082617,0.162602,-0.069705,0.21748,0.359248
2,AAP,75.14,N/A,-0.376010,0.018293,0.483286,0.896341,0.137838,0.823171,-0.105261,0.101626,0.459858
3,AAPL,169.30,N/A,0.043452,0.371951,0.019712,0.184959,-0.117907,0.105691,-0.002416,0.611789,0.318598
4,ABBV,159.62,N/A,0.026571,0.337398,0.120179,0.367886,-0.020008,0.315041,-0.100894,0.115854,0.284045
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,141.77,N/A,0.046991,0.373984,0.205503,0.575203,0.103722,0.754065,0.051706,0.896341,0.649898
492,ZBH,119.35,N/A,-0.135226,0.130081,0.161878,0.471545,-0.017385,0.325203,-0.064875,0.235772,0.29065
493,ZBRA,297.21,N/A,0.059874,0.398374,0.451008,0.871951,0.178610,0.894309,0.020008,0.756098,0.730183
494,ZION,41.76,N/A,0.666082,0.932927,0.402736,0.835366,-0.041349,0.256098,0.013838,0.707317,0.682927


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False,inplace=True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop=True,inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,GE,162.35,N/A,1.127766,0.98374,0.878805,0.997967,0.557175,1.0,0.177101,0.99187,0.993394
1,NRG,72.81,N/A,1.263802,0.989837,0.754141,0.985772,0.352320,0.99187,0.105191,0.977642,0.98628
2,LB,79.92,N/A,2.281287,1.0,0.815831,0.993902,0.215421,0.947154,0.079271,0.957317,0.974593
3,CMG,3186.97,N/A,0.585676,0.912602,0.764198,0.989837,0.370003,0.995935,0.078307,0.953252,0.962907
4,WAB,164.36,N/A,0.713767,0.945122,0.577359,0.939024,0.266364,0.971545,0.142103,0.985772,0.960366
5,WDC,71.36,N/A,1.183599,0.985772,0.865133,0.995935,0.225485,0.955285,0.047102,0.878049,0.95376
6,ETN,324.30,N/A,1.035553,0.979675,0.661740,0.97561,0.323013,0.98374,0.039157,0.851626,0.947663
7,DAL,49.92,N/A,0.535395,0.878049,0.583505,0.945122,0.262650,0.969512,0.086871,0.969512,0.940549
8,APH,120.49,N/A,0.645382,0.922764,0.508743,0.912602,0.198818,0.920732,0.063085,0.928862,0.92124
9,WFC,59.91,N/A,0.566766,0.892276,0.535150,0.926829,0.198943,0.924797,0.056987,0.916667,0.915142


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the size of your portfolio: 10000000


In [15]:
position_size =float(portfolio_size)/len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,GE,162.35,1207,1.127766,0.98374,0.878805,0.997967,0.557175,1.0,0.177101,0.99187,0.993394
1,NRG,72.81,2693,1.263802,0.989837,0.754141,0.985772,0.352320,0.99187,0.105191,0.977642,0.98628
2,LB,79.92,2453,2.281287,1.0,0.815831,0.993902,0.215421,0.947154,0.079271,0.957317,0.974593
3,CMG,3186.97,61,0.585676,0.912602,0.764198,0.989837,0.370003,0.995935,0.078307,0.953252,0.962907
4,WAB,164.36,1192,0.713767,0.945122,0.577359,0.939024,0.266364,0.971545,0.142103,0.985772,0.960366
5,WDC,71.36,2747,1.183599,0.985772,0.865133,0.995935,0.225485,0.955285,0.047102,0.878049,0.95376
6,ETN,324.30,604,1.035553,0.979675,0.661740,0.97561,0.323013,0.98374,0.039157,0.851626,0.947663
7,DAL,49.92,3927,0.535395,0.878049,0.583505,0.945122,0.262650,0.969512,0.086871,0.969512,0.940549
8,APH,120.49,1627,0.645382,0.922764,0.508743,0.912602,0.198818,0.920732,0.063085,0.928862,0.92124
9,WFC,59.91,3272,0.566766,0.892276,0.535150,0.926829,0.198943,0.924797,0.056987,0.916667,0.915142


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.close()